In [45]:
import pandas as pd

In [46]:
predictions = pd.read_csv(f"../data/predictions/predictions-{11}-{19}.csv")
final_predictions = pd.read_csv(f"../data/predictions/predictions-final-{11}-{17}.csv")

In [47]:
features = pd.read_csv("../data/tsfeatures/tsfeatures.csv")\
    .dropna(axis = 1, how = 'all')

features = features.rename({"unique_id":"store_item"}, axis=1)

In [48]:
final_feats = pd.read_csv(f"../data/tsfeatures/tsfeatures_final.csv")\
    .dropna(axis = 1, how = 'all')
    
final_feats = final_feats.rename({"unique_id":"store_item"}, axis=1)

In [49]:
from sklearn.impute import KNNImputer

def impute_hurst(df):
    
    imputer = KNNImputer()
    knn_impute = df.select_dtypes(include=['float64'])
    knn_imputed = imputer.fit_transform(knn_impute)
    knn_imputed = pd.DataFrame(knn_imputed, columns=knn_impute.columns)
    return knn_imputed[["hurst", "entropy"]]

features[["hurst", "entropy"]] = impute_hurst(features)
final_feats[["hurst", "entropy"]] = impute_hurst(final_feats)

In [50]:
def drop_same_val_colums(df):
    nunique = df.nunique()
    cols_to_drop = nunique[nunique == 1].index
    return df.drop(cols_to_drop, axis=1)

features = drop_same_val_colums(features)
final_feats = drop_same_val_colums(final_feats)

In [51]:
predictions = predictions.rename({'Unnamed: 0':"date"}, axis=1)
predictions.date = pd.to_datetime(predictions.date)

final_predictions = final_predictions.rename({'Unnamed: 0':"date"}, axis=1)
final_predictions.date = pd.to_datetime(final_predictions.date)

In [52]:
predictions.to_csv("../data/predictions/errors.csv", index=False)

In [53]:
predictions

,date,store_item,sales,autoreg,exp_smooth,ardl,xgb_preds,prophet,neural_prophet
0,2016-07-03,1-1,35,33.967322,34.968360,28.270421,26.026503,31.430007,32.687370
1,2016-07-04,1-1,18,27.360797,27.868874,21.079731,11.948491,23.552477,24.825729
2,2016-07-05,1-1,31,26.177283,26.861003,21.245674,9.154426,26.190409,27.409180
3,2016-07-06,1-1,22,27.303443,28.344893,27.543148,10.446870,26.846605,28.147953
4,2016-07-07,1-1,33,23.602249,25.971329,29.099083,11.971621,27.095248,28.449118
...,...,...,...,...,...,...,...,...,...
273495,2017-12-27,10-50,63,54.000000,70.629780,52.203080,53.286358,60.990158,76.731506
273496,2017-12-28,10-50,59,48.333333,64.050097,60.144470,61.416860,64.923885,80.565811
273497,2017-12-29,10-50,74,41.333333,56.527002,60.437527,62.223686,69.576875,85.233772
273498,2017-12-30,10-50,62,43.666667,59.522268,69.928363,67.933410,74.064814,89.705673


In [54]:
error = predictions.iloc[:, 3:9] # fcast_models
predictions.drop(predictions.iloc[:, 3:9], inplace=True, axis=1)
errors = error.sub(predictions.sales, axis=0)

In [55]:
from helper import create_features

In [56]:
final_predictions = pd.concat([final_predictions, create_features(final_predictions)], axis=1)
predictions = pd.concat([predictions, create_features(predictions)], axis=1)

In [57]:
final_df = pd.merge(predictions, features, right_on="store_item", left_on= "store_item")
y = errors.abs().idxmin(axis=1)
X = final_df.drop(final_df.columns[[0, 1, 2, 19]], inplace=False, axis=1)
X = X.loc[:,~X.columns.duplicated()]

In [58]:
y.to_csv("../data/forecasts_for_ml/y.csv", index=False)
X.to_csv("../data/forecasts_for_ml/X.csv", index=False)

In [59]:
final_predictions_df = pd.merge(final_predictions, final_feats, right_on="store_item", left_on= "store_item")
final_predictions_df = final_predictions_df[X.columns]
final_predictions_df = final_predictions_df.loc[:,~final_predictions_df.columns.duplicated()]
final_predictions_df.to_csv("../data/forecasts_for_ml/X_final.csv", index=False)